In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer
    
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay1")

okay1


In [20]:
for k in [1..10]:
    rt=rmnjntau(3^k)
    ort=order(3,rt)
    print((k,ort))

(1, 2)
(2, 4)
(3, 6)
(4, 8)
(5, 10)
(6, 12)
(7, 14)
(8, 16)
(9, 18)
(10, 20)


In [12]:
import time
import random

def specialModulus(a):
    ds=digitsum(3,a)
    return 3*3^ds

def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        
        d=a^2*sigma(a,k=7)
        
        term=d  
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+1)
    gl=g.list()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1010)
zeros=[]
for b in [1..1]:
    tests=[]
    for a in [1..20]:
        c=recipPowerConstantTerm(a)
        if c==0:zeros=zeros+[a]
        if c!=0:
            o=order(3,c)
            ds=digitsum(3,a)
            test=ds-o
            tests=tests+[test]
            print((a,test))
    print(tests);print();print(zeros)

(1, 0)
(2, 0)
(3, 0)
(4, 0)
(5, 0)
(6, 0)
(7, 0)
(8, 0)
(9, 0)
(10, 0)
(11, 0)
(12, 0)
(13, 0)
(14, 0)
(15, 0)
(16, 0)
(17, 0)
(18, 0)
(19, 0)
(20, 0)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

[]


In [15]:
import time

def specialModulus(a):
    ds=digitsum(3,a)
    return 3*3^ds

def RamanujanCongruenceSeries(n): 
    ans=0
    
    for a in [1..n+1]:
       
        
        d=a^2*sigma(a,k=7)
        
        term=d  
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+1)
    gl=g.list()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)
zeros=[]
for b in [1..1]:
    tests=[]
    mds=[]
    for a in [1..2000]:
        c=recipPowerConstantTerm(a)
        if c==0:zeros=zeros+[a]
        if c!=0:
            o=order(3,c)
            ds=digitsum(3,a)
            test=ds-o
            tests=tests+[test]
            sk=c/3^o
            md=mod(sk,3)
            mds=mds+[md]
            print((a,test,md))
    print(tests);print();print(zeros);print();print(mds)

(1, 0, 2)
(2, 0, 1)
(3, 0, 2)
(4, 0, 1)
(5, 0, 2)
(6, 0, 1)
(7, 0, 2)
(8, 0, 1)
(9, 0, 2)
(10, 0, 1)
(11, 0, 2)
(12, 0, 1)
(13, 0, 2)
(14, 0, 1)
(15, 0, 2)
(16, 0, 1)
(17, 0, 2)
(18, 0, 1)
(19, 0, 2)
(20, 0, 1)
(21, 0, 2)
(22, 0, 1)
(23, 0, 2)
(24, 0, 1)
(25, 0, 2)
(26, 0, 1)
(27, 0, 2)
(28, 0, 1)
(29, 0, 2)
(30, 0, 1)
(31, 0, 2)
(32, 0, 1)
(33, 0, 2)
(34, 0, 1)
(35, 0, 2)
(36, 0, 1)
(37, 0, 2)
(38, 0, 1)
(39, 0, 2)
(40, 0, 1)
(41, 0, 2)
(42, 0, 1)
(43, 0, 2)
(44, 0, 1)
(45, 0, 2)
(46, 0, 1)
(47, 0, 2)
(48, 0, 1)
(49, 0, 2)
(50, 0, 1)
(51, 0, 2)
(52, 0, 1)
(53, 0, 2)
(54, 0, 1)
(55, 0, 2)
(56, 0, 1)
(57, 0, 2)
(58, 0, 1)
(59, 0, 2)
(60, 0, 1)
(61, 0, 2)
(62, 0, 1)
(63, 0, 2)
(64, 0, 1)
(65, 0, 2)
(66, 0, 1)
(67, 0, 2)
(68, 0, 1)
(69, 0, 2)
(70, 0, 1)
(71, 0, 2)
(72, 0, 1)
(73, 0, 2)
(74, 0, 1)
(75, 0, 2)
(76, 0, 1)
(77, 0, 2)
(78, 0, 1)
(79, 0, 2)
(80, 0, 1)
(81, 0, 2)
(82, 0, 1)
(83, 0, 2)
(84, 0, 1)
(85, 0, 2)
(86, 0, 1)
(87, 0, 2)
(88, 0, 1)
(89, 0, 2)
(90, 0, 1)
(91, 0, 2)
(92, 0, 

(693, 0, 2)
(694, 0, 1)
(695, 0, 2)
(696, 0, 1)
(697, 0, 2)
(698, 0, 1)
(699, 0, 2)
(700, 0, 1)
(701, 0, 2)
(702, 0, 1)
(703, 0, 2)
(704, 0, 1)
(705, 0, 2)
(706, 0, 1)
(707, 0, 2)
(708, 0, 1)
(709, 0, 2)
(710, 0, 1)
(711, 0, 2)
(712, 0, 1)
(713, 0, 2)
(714, 0, 1)
(715, 0, 2)
(716, 0, 1)
(717, 0, 2)
(718, 0, 1)
(719, 0, 2)
(720, 0, 1)
(721, 0, 2)
(722, 0, 1)
(723, 0, 2)
(724, 0, 1)
(725, 0, 2)
(726, 0, 1)
(727, 0, 2)
(728, 0, 1)
(729, 0, 2)
(730, 0, 1)
(731, 0, 2)
(732, 0, 1)
(733, 0, 2)
(734, 0, 1)
(735, 0, 2)
(736, 0, 1)
(737, 0, 2)
(738, 0, 1)
(739, 0, 2)
(740, 0, 1)
(741, 0, 2)
(742, 0, 1)
(743, 0, 2)
(744, 0, 1)
(745, 0, 2)
(746, 0, 1)
(747, 0, 2)
(748, 0, 1)
(749, 0, 2)
(750, 0, 1)
(751, 0, 2)
(752, 0, 1)
(753, 0, 2)
(754, 0, 1)
(755, 0, 2)
(756, 0, 1)
(757, 0, 2)
(758, 0, 1)
(759, 0, 2)
(760, 0, 1)
(761, 0, 2)
(762, 0, 1)
(763, 0, 2)
(764, 0, 1)
(765, 0, 2)
(766, 0, 1)
(767, 0, 2)
(768, 0, 1)
(769, 0, 2)
(770, 0, 1)
(771, 0, 2)
(772, 0, 1)
(773, 0, 2)
(774, 0, 1)
(775, 0, 2)
(776

(1347, 0, 2)
(1348, 0, 1)
(1349, 0, 2)
(1350, 0, 1)
(1351, 0, 2)
(1352, 0, 1)
(1353, 0, 2)
(1354, 0, 1)
(1355, 0, 2)
(1356, 0, 1)
(1357, 0, 2)
(1358, 0, 1)
(1359, 0, 2)
(1360, 0, 1)
(1361, 0, 2)
(1362, 0, 1)
(1363, 0, 2)
(1364, 0, 1)
(1365, 0, 2)
(1366, 0, 1)
(1367, 0, 2)
(1368, 0, 1)
(1369, 0, 2)
(1370, 0, 1)
(1371, 0, 2)
(1372, 0, 1)
(1373, 0, 2)
(1374, 0, 1)
(1375, 0, 2)
(1376, 0, 1)
(1377, 0, 2)
(1378, 0, 1)
(1379, 0, 2)
(1380, 0, 1)
(1381, 0, 2)
(1382, 0, 1)
(1383, 0, 2)
(1384, 0, 1)
(1385, 0, 2)
(1386, 0, 1)
(1387, 0, 2)
(1388, 0, 1)
(1389, 0, 2)
(1390, 0, 1)
(1391, 0, 2)
(1392, 0, 1)
(1393, 0, 2)
(1394, 0, 1)
(1395, 0, 2)
(1396, 0, 1)
(1397, 0, 2)
(1398, 0, 1)
(1399, 0, 2)
(1400, 0, 1)
(1401, 0, 2)
(1402, 0, 1)
(1403, 0, 2)
(1404, 0, 1)
(1405, 0, 2)
(1406, 0, 1)
(1407, 0, 2)
(1408, 0, 1)
(1409, 0, 2)
(1410, 0, 1)
(1411, 0, 2)
(1412, 0, 1)
(1413, 0, 2)
(1414, 0, 1)
(1415, 0, 2)
(1416, 0, 1)
(1417, 0, 2)
(1418, 0, 1)
(1419, 0, 2)
(1420, 0, 1)
(1421, 0, 2)
(1422, 0, 1)
(1423, 0, 2)

(1978, 0, 1)
(1979, 0, 2)
(1980, 0, 1)
(1981, 0, 2)
(1982, 0, 1)
(1983, 0, 2)
(1984, 0, 1)
(1985, 0, 2)
(1986, 0, 1)
(1987, 0, 2)
(1988, 0, 1)
(1989, 0, 2)
(1990, 0, 1)
(1991, 0, 2)
(1992, 0, 1)
(1993, 0, 2)
(1994, 0, 1)
(1995, 0, 2)
(1996, 0, 1)
(1997, 0, 2)
(1998, 0, 1)
(1999, 0, 2)
(2000, 0, 1)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [20]:
import time
import random

def specialModulus(a):
    ds=digitsum(3,a)
    return 3*3^ds

def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        
        d=a^2*sigma(a,k=7)
        
        term=d+sm*r      
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+1)
    gl=g.list()
    ln=len(gl)
    return gl[ln-2]
    
bound=1000
L=LaurentSeriesRing(QQ,'x', default_prec=bound+10)

for b in [1..3]:
    tests=[]
    zeros=[]
    mds=[]
    for a in [1..bound]:
        c=recipPowerConstantTerm(a)
        if c==0:zeros=zeros+[a]
        if c!=0:
            o=order(3,c)
            sk=c/3^o
            md=ZZ(mod(sk,3)-(-1)^a) # easier to check by eye if it's all zeros when
            # the hypothesis holds.
            mds=mds+[md]
            ds=digitsum(3,a)
            test=ds-o
            tests=tests+[test]
            print((b,a,test,mod(sk,3),md))
    print(tests);print();print(zeros);print();print(mds)

(1, 1, 0, 2, 0)
(1, 2, 0, 1, 0)
(1, 3, 0, 2, 0)
(1, 4, 0, 1, 0)
(1, 5, 0, 2, 0)
(1, 6, 0, 1, 0)
(1, 7, 0, 2, 0)
(1, 8, 0, 1, 0)
(1, 9, 0, 2, 0)
(1, 10, 0, 1, 0)
(1, 11, 0, 2, 0)
(1, 12, 0, 1, 0)
(1, 13, 0, 2, 0)
(1, 14, 0, 1, 0)
(1, 15, 0, 2, 0)
(1, 16, 0, 1, 0)
(1, 17, 0, 2, 0)
(1, 18, 0, 1, 0)
(1, 19, 0, 2, 0)
(1, 20, 0, 1, 0)
(1, 21, 0, 2, 0)
(1, 22, 0, 1, 0)
(1, 23, 0, 2, 0)
(1, 24, 0, 1, 0)
(1, 25, 0, 2, 0)
(1, 26, 0, 1, 0)
(1, 27, 0, 2, 0)
(1, 28, 0, 1, 0)
(1, 29, 0, 2, 0)
(1, 30, 0, 1, 0)
(1, 31, 0, 2, 0)
(1, 32, 0, 1, 0)
(1, 33, 0, 2, 0)
(1, 34, 0, 1, 0)
(1, 35, 0, 2, 0)
(1, 36, 0, 1, 0)
(1, 37, 0, 2, 0)
(1, 38, 0, 1, 0)
(1, 39, 0, 2, 0)
(1, 40, 0, 1, 0)
(1, 41, 0, 2, 0)
(1, 42, 0, 1, 0)
(1, 43, 0, 2, 0)
(1, 44, 0, 1, 0)
(1, 45, 0, 2, 0)
(1, 46, 0, 1, 0)
(1, 47, 0, 2, 0)
(1, 48, 0, 1, 0)
(1, 49, 0, 2, 0)
(1, 50, 0, 1, 0)
(1, 51, 0, 2, 0)
(1, 52, 0, 1, 0)
(1, 53, 0, 2, 0)
(1, 54, 0, 1, 0)
(1, 55, 0, 2, 0)
(1, 56, 0, 1, 0)
(1, 57, 0, 2, 0)
(1, 58, 0, 1, 0)
(1, 59, 0, 2, 0)
(1, 60

(1, 463, 0, 2, 0)
(1, 464, 0, 1, 0)
(1, 465, 0, 2, 0)
(1, 466, 0, 1, 0)
(1, 467, 0, 2, 0)
(1, 468, 0, 1, 0)
(1, 469, 0, 2, 0)
(1, 470, 0, 1, 0)
(1, 471, 0, 2, 0)
(1, 472, 0, 1, 0)
(1, 473, 0, 2, 0)
(1, 474, 0, 1, 0)
(1, 475, 0, 2, 0)
(1, 476, 0, 1, 0)
(1, 477, 0, 2, 0)
(1, 478, 0, 1, 0)
(1, 479, 0, 2, 0)
(1, 480, 0, 1, 0)
(1, 481, 0, 2, 0)
(1, 482, 0, 1, 0)
(1, 483, 0, 2, 0)
(1, 484, 0, 1, 0)
(1, 485, 0, 2, 0)
(1, 486, 0, 1, 0)
(1, 487, 0, 2, 0)
(1, 488, 0, 1, 0)
(1, 489, 0, 2, 0)
(1, 490, 0, 1, 0)
(1, 491, 0, 2, 0)
(1, 492, 0, 1, 0)
(1, 493, 0, 2, 0)
(1, 494, 0, 1, 0)
(1, 495, 0, 2, 0)
(1, 496, 0, 1, 0)
(1, 497, 0, 2, 0)
(1, 498, 0, 1, 0)
(1, 499, 0, 2, 0)
(1, 500, 0, 1, 0)
(1, 501, 0, 2, 0)
(1, 502, 0, 1, 0)
(1, 503, 0, 2, 0)
(1, 504, 0, 1, 0)
(1, 505, 0, 2, 0)
(1, 506, 0, 1, 0)
(1, 507, 0, 2, 0)
(1, 508, 0, 1, 0)
(1, 509, 0, 2, 0)
(1, 510, 0, 1, 0)
(1, 511, 0, 2, 0)
(1, 512, 0, 1, 0)
(1, 513, 0, 2, 0)
(1, 514, 0, 1, 0)
(1, 515, 0, 2, 0)
(1, 516, 0, 1, 0)
(1, 517, 0, 2, 0)
(1, 518, 0

(1, 919, 0, 2, 0)
(1, 920, 0, 1, 0)
(1, 921, 0, 2, 0)
(1, 922, 0, 1, 0)
(1, 923, 0, 2, 0)
(1, 924, 0, 1, 0)
(1, 925, 0, 2, 0)
(1, 926, 0, 1, 0)
(1, 927, 0, 2, 0)
(1, 928, 0, 1, 0)
(1, 929, 0, 2, 0)
(1, 930, 0, 1, 0)
(1, 931, 0, 2, 0)
(1, 932, 0, 1, 0)
(1, 933, 0, 2, 0)
(1, 934, 0, 1, 0)
(1, 935, 0, 2, 0)
(1, 936, 0, 1, 0)
(1, 937, 0, 2, 0)
(1, 938, 0, 1, 0)
(1, 939, 0, 2, 0)
(1, 940, 0, 1, 0)
(1, 941, 0, 2, 0)
(1, 942, 0, 1, 0)
(1, 943, 0, 2, 0)
(1, 944, 0, 1, 0)
(1, 945, 0, 2, 0)
(1, 946, 0, 1, 0)
(1, 947, 0, 2, 0)
(1, 948, 0, 1, 0)
(1, 949, 0, 2, 0)
(1, 950, 0, 1, 0)
(1, 951, 0, 2, 0)
(1, 952, 0, 1, 0)
(1, 953, 0, 2, 0)
(1, 954, 0, 1, 0)
(1, 955, 0, 2, 0)
(1, 956, 0, 1, 0)
(1, 957, 0, 2, 0)
(1, 958, 0, 1, 0)
(1, 959, 0, 2, 0)
(1, 960, 0, 1, 0)
(1, 961, 0, 2, 0)
(1, 962, 0, 1, 0)
(1, 963, 0, 2, 0)
(1, 964, 0, 1, 0)
(1, 965, 0, 2, 0)
(1, 966, 0, 1, 0)
(1, 967, 0, 2, 0)
(1, 968, 0, 1, 0)
(1, 969, 0, 2, 0)
(1, 970, 0, 1, 0)
(1, 971, 0, 2, 0)
(1, 972, 0, 1, 0)
(1, 973, 0, 2, 0)
(1, 974, 0

(2, 45, 0, 2, 0)
(2, 46, 0, 1, 0)
(2, 47, 0, 2, 0)
(2, 48, 0, 1, 0)
(2, 49, 0, 2, 0)
(2, 50, 0, 1, 0)
(2, 51, 0, 2, 0)
(2, 52, 0, 1, 0)
(2, 53, 0, 2, 0)
(2, 54, 0, 1, 0)
(2, 55, 0, 2, 0)
(2, 56, 0, 1, 0)
(2, 57, 0, 2, 0)
(2, 58, 0, 1, 0)
(2, 59, 0, 2, 0)
(2, 60, 0, 1, 0)
(2, 61, 0, 2, 0)
(2, 62, 0, 1, 0)
(2, 63, 0, 2, 0)
(2, 64, 0, 1, 0)
(2, 65, 0, 2, 0)
(2, 66, 0, 1, 0)
(2, 67, 0, 2, 0)
(2, 68, 0, 1, 0)
(2, 69, 0, 2, 0)
(2, 70, 0, 1, 0)
(2, 71, 0, 2, 0)
(2, 72, 0, 1, 0)
(2, 73, 0, 2, 0)
(2, 74, 0, 1, 0)
(2, 75, 0, 2, 0)
(2, 76, 0, 1, 0)
(2, 77, 0, 2, 0)
(2, 78, 0, 1, 0)
(2, 79, 0, 2, 0)
(2, 80, 0, 1, 0)
(2, 81, 0, 2, 0)
(2, 82, 0, 1, 0)
(2, 83, 0, 2, 0)
(2, 84, 0, 1, 0)
(2, 85, 0, 2, 0)
(2, 86, 0, 1, 0)
(2, 87, 0, 2, 0)
(2, 88, 0, 1, 0)
(2, 89, 0, 2, 0)
(2, 90, 0, 1, 0)
(2, 91, 0, 2, 0)
(2, 92, 0, 1, 0)
(2, 93, 0, 2, 0)
(2, 94, 0, 1, 0)
(2, 95, 0, 2, 0)
(2, 96, 0, 1, 0)
(2, 97, 0, 2, 0)
(2, 98, 0, 1, 0)
(2, 99, 0, 2, 0)
(2, 100, 0, 1, 0)
(2, 101, 0, 2, 0)
(2, 102, 0, 1, 0)
(2, 103, 0,

(2, 504, 0, 1, 0)
(2, 505, 0, 2, 0)
(2, 506, 0, 1, 0)
(2, 507, 0, 2, 0)
(2, 508, 0, 1, 0)
(2, 509, 0, 2, 0)
(2, 510, 0, 1, 0)
(2, 511, 0, 2, 0)
(2, 512, 0, 1, 0)
(2, 513, 0, 2, 0)
(2, 514, 0, 1, 0)
(2, 515, 0, 2, 0)
(2, 516, 0, 1, 0)
(2, 517, 0, 2, 0)
(2, 518, 0, 1, 0)
(2, 519, 0, 2, 0)
(2, 520, 0, 1, 0)
(2, 521, 0, 2, 0)
(2, 522, 0, 1, 0)
(2, 523, 0, 2, 0)
(2, 524, 0, 1, 0)
(2, 525, 0, 2, 0)
(2, 526, 0, 1, 0)
(2, 527, 0, 2, 0)
(2, 528, 0, 1, 0)
(2, 529, 0, 2, 0)
(2, 530, 0, 1, 0)
(2, 531, 0, 2, 0)
(2, 532, 0, 1, 0)
(2, 533, 0, 2, 0)
(2, 534, 0, 1, 0)
(2, 535, 0, 2, 0)
(2, 536, 0, 1, 0)
(2, 537, 0, 2, 0)
(2, 538, 0, 1, 0)
(2, 539, 0, 2, 0)
(2, 540, 0, 1, 0)
(2, 541, 0, 2, 0)
(2, 542, 0, 1, 0)
(2, 543, 0, 2, 0)
(2, 544, 0, 1, 0)
(2, 545, 0, 2, 0)
(2, 546, 0, 1, 0)
(2, 547, 0, 2, 0)
(2, 548, 0, 1, 0)
(2, 549, 0, 2, 0)
(2, 550, 0, 1, 0)
(2, 551, 0, 2, 0)
(2, 552, 0, 1, 0)
(2, 553, 0, 2, 0)
(2, 554, 0, 1, 0)
(2, 555, 0, 2, 0)
(2, 556, 0, 1, 0)
(2, 557, 0, 2, 0)
(2, 558, 0, 1, 0)
(2, 559, 0

(2, 960, 0, 1, 0)
(2, 961, 0, 2, 0)
(2, 962, 0, 1, 0)
(2, 963, 0, 2, 0)
(2, 964, 0, 1, 0)
(2, 965, 0, 2, 0)
(2, 966, 0, 1, 0)
(2, 967, 0, 2, 0)
(2, 968, 0, 1, 0)
(2, 969, 0, 2, 0)
(2, 970, 0, 1, 0)
(2, 971, 0, 2, 0)
(2, 972, 0, 1, 0)
(2, 973, 0, 2, 0)
(2, 974, 0, 1, 0)
(2, 975, 0, 2, 0)
(2, 976, 0, 1, 0)
(2, 977, 0, 2, 0)
(2, 978, 0, 1, 0)
(2, 979, 0, 2, 0)
(2, 980, 0, 1, 0)
(2, 981, 0, 2, 0)
(2, 982, 0, 1, 0)
(2, 983, 0, 2, 0)
(2, 984, 0, 1, 0)
(2, 985, 0, 2, 0)
(2, 986, 0, 1, 0)
(2, 987, 0, 2, 0)
(2, 988, 0, 1, 0)
(2, 989, 0, 2, 0)
(2, 990, 0, 1, 0)
(2, 991, 0, 2, 0)
(2, 992, 0, 1, 0)
(2, 993, 0, 2, 0)
(2, 994, 0, 1, 0)
(2, 995, 0, 2, 0)
(2, 996, 0, 1, 0)
(2, 997, 0, 2, 0)
(2, 998, 0, 1, 0)
(2, 999, 0, 2, 0)
(2, 1000, 0, 1, 0)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

(3, 87, 0, 2, 0)
(3, 88, 0, 1, 0)
(3, 89, 0, 2, 0)
(3, 90, 0, 1, 0)
(3, 91, 0, 2, 0)
(3, 92, 0, 1, 0)
(3, 93, 0, 2, 0)
(3, 94, 0, 1, 0)
(3, 95, 0, 2, 0)
(3, 96, 0, 1, 0)
(3, 97, 0, 2, 0)
(3, 98, 0, 1, 0)
(3, 99, 0, 2, 0)
(3, 100, 0, 1, 0)
(3, 101, 0, 2, 0)
(3, 102, 0, 1, 0)
(3, 103, 0, 2, 0)
(3, 104, 0, 1, 0)
(3, 105, 0, 2, 0)
(3, 106, 0, 1, 0)
(3, 107, 0, 2, 0)
(3, 108, 0, 1, 0)
(3, 109, 0, 2, 0)
(3, 110, 0, 1, 0)
(3, 111, 0, 2, 0)
(3, 112, 0, 1, 0)
(3, 113, 0, 2, 0)
(3, 114, 0, 1, 0)
(3, 115, 0, 2, 0)
(3, 116, 0, 1, 0)
(3, 117, 0, 2, 0)
(3, 118, 0, 1, 0)
(3, 119, 0, 2, 0)
(3, 120, 0, 1, 0)
(3, 121, 0, 2, 0)
(3, 122, 0, 1, 0)
(3, 123, 0, 2, 0)
(3, 124, 0, 1, 0)
(3, 125, 0, 2, 0)
(3, 126, 0, 1, 0)
(3, 127, 0, 2, 0)
(3, 128, 0, 1, 0)
(3, 129, 0, 2, 0)
(3, 130, 0, 1, 0)
(3, 131, 0, 2, 0)
(3, 132, 0, 1, 0)
(3, 133, 0, 2, 0)
(3, 134, 0, 1, 0)
(3, 135, 0, 2, 0)
(3, 136, 0, 1, 0)
(3, 137, 0, 2, 0)
(3, 138, 0, 1, 0)
(3, 139, 0, 2, 0)
(3, 140, 0, 1, 0)
(3, 141, 0, 2, 0)
(3, 142, 0, 1, 0)
(3, 1

(3, 543, 0, 2, 0)
(3, 544, 0, 1, 0)
(3, 545, 0, 2, 0)
(3, 546, 0, 1, 0)
(3, 547, 0, 2, 0)
(3, 548, 0, 1, 0)
(3, 549, 0, 2, 0)
(3, 550, 0, 1, 0)
(3, 551, 0, 2, 0)
(3, 552, 0, 1, 0)
(3, 553, 0, 2, 0)
(3, 554, 0, 1, 0)
(3, 555, 0, 2, 0)
(3, 556, 0, 1, 0)
(3, 557, 0, 2, 0)
(3, 558, 0, 1, 0)
(3, 559, 0, 2, 0)
(3, 560, 0, 1, 0)
(3, 561, 0, 2, 0)
(3, 562, 0, 1, 0)
(3, 563, 0, 2, 0)
(3, 564, 0, 1, 0)
(3, 565, 0, 2, 0)
(3, 566, 0, 1, 0)
(3, 567, 0, 2, 0)
(3, 568, 0, 1, 0)
(3, 569, 0, 2, 0)
(3, 570, 0, 1, 0)
(3, 571, 0, 2, 0)
(3, 572, 0, 1, 0)
(3, 573, 0, 2, 0)
(3, 574, 0, 1, 0)
(3, 575, 0, 2, 0)
(3, 576, 0, 1, 0)
(3, 577, 0, 2, 0)
(3, 578, 0, 1, 0)
(3, 579, 0, 2, 0)
(3, 580, 0, 1, 0)
(3, 581, 0, 2, 0)
(3, 582, 0, 1, 0)
(3, 583, 0, 2, 0)
(3, 584, 0, 1, 0)
(3, 585, 0, 2, 0)
(3, 586, 0, 1, 0)
(3, 587, 0, 2, 0)
(3, 588, 0, 1, 0)
(3, 589, 0, 2, 0)
(3, 590, 0, 1, 0)
(3, 591, 0, 2, 0)
(3, 592, 0, 1, 0)
(3, 593, 0, 2, 0)
(3, 594, 0, 1, 0)
(3, 595, 0, 2, 0)
(3, 596, 0, 1, 0)
(3, 597, 0, 2, 0)
(3, 598, 0

(3, 999, 0, 2, 0)
(3, 1000, 0, 1, 0)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,